In [64]:
import pandas as pd
import os
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [65]:
working_dir = os.getcwd()
rel_path = r"project_resources\chembl_molekuly.csv"
molekuly_csv_abs_file_path = os.path.join(working_dir, rel_path)
#loads csv file from directory

In [66]:
#dir for useful stuff for the actual essay
rel_path = r"project_results\graphs"
project_results_graphs = os.path.join(working_dir, rel_path)

In [67]:
df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")
#df.head()

C:\Users\Lukas\AppData\Local\Temp\ipykernel_19880\3345623085.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")


In [68]:
df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")
statistika = df["Standard Type"].value_counts()
#print(statistika)

C:\Users\Lukas\AppData\Local\Temp\ipykernel_19880\4010148655.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(molekuly_csv_abs_file_path, sep=";")


In [69]:
result = df[(df["Standard Type"] == "t1/2") | (df["Standard Type"] == "T1/2")]
t_half_rel_path = r"project_resources\t_half.csv"
t_half_abs_path = os.path.join(working_dir, t_half_rel_path)
result.to_csv(t_half_abs_path, index=False)
#creates "t_half.csv" in current dir
t_half = pd.read_csv(t_half_abs_path)
#and then loads it

In [70]:
client = new_client
list_smiles = []
list_chembl_id = t_half["Molecule ChEMBL ID"]
for chembl_id in list_chembl_id:
    molecule = client.molecule
    compound = molecule.filter(chembl_id=chembl_id)[0]
    list_smiles.append(compound['molecule_structures']["canonical_smiles"])
#print(list_smiles)

In [71]:
#takes a list of smiles strings,output is a corresponding Morgan fingerprint as a list
list_fingerprint = []
for smi in list_smiles:
    mol = Chem.MolFromSmiles(smi)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, useChirality=True, radius=2, nBits = 124)
    vector = np.array(fingerprint)
    list_fingerprint.append(vector)
    #print(list_fingerprint)

In [72]:
array_fingerprint = np.array(list_fingerprint)
array_fingerprint.shape

(74, 124)

In [73]:
t_half = t_half['Standard Value']
t_half.shape

(74,)

In [74]:
t_filtered = t_half[t_half>0]
t_filtered.shape

(70,)

In [75]:
t_half = t_half.astype(float)
t_filtered = t_half[t_half>0]
array_fingerprint_filtered = array_fingerprint[t_half>0]
# x su len fingerprint ako jeden numpy array,
# y su len t_half hodnoty ako jeden numpy array, pouzit len molekuly kde existuje aj X aj y hodnota
x = array_fingerprint_filtered
y = t_filtered
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(56, 124) (14, 124) (56,) (14,)


In [76]:
def model_choice(ml_type):
    
    if ml_type == 'linear':
        reg = linear_model.ElasticNet(alpha=0.001, l1_ratio=1)
        plot_title = "Linear"
        
    if ml_type == 'KRR':
        reg = KernelRidge(alpha=0.1)
        plot_title = "Kernel ridge regressor"
        
    if ml_type == 'RF':
        reg = RandomForestRegressor(max_depth=None, random_state=0)
        plot_title = "Random forest regressor"
        
    if ml_type == 'GB':
        reg = GradientBoostingRegressor(random_state=1, loss='absolute_error', max_depth=5, n_estimators=1000)
        plot_title = "Gradient boosting regressor"
        
    if ml_type == 'ANN':
        reg = MLPRegressor(random_state=1, max_iter=5000, learning_rate_init=0.001, alpha=0, hidden_layer_sizes=[10,10])
        plot_title = "Limited-memory BFGS"
    print(f"using ML type {ml_type}")
    
    return reg, plot_title

In [77]:
def mach_learn(x_train, x_test, y_train, y_test, reg, plot_title):
    # výcvik strojoveho modelu
    reg = reg.fit(x_train, y_train)
    # predict training set
    y_train_predict = reg.predict(x_train)
    # predict test set
    y_test_predict = reg.predict(x_test)
    #error train je pre info, test je dolezita
    train_error = np.mean(np.abs(y_train_predict-y_train))
    test_error = np.mean(np.abs(y_test_predict-y_test))
    print(f"MAE train: {train_error:.4f}")
    print(f"MAE test: {test_error:.4f}")

    # idealne data na diagonale
    diag = np.linspace(-1, 3.5)

    #plot = plt.scatter(y_test,y_test_predict,edgecolors=None,c='b',alpha=0.2)
    #plt.plot(diag, diag, linestyle='dotted')
    #plt.xlim(-1, 3.5)
    #plt.ylim(-1, 3.5)
    #plt.ylabel('$t_{1/2}$ (predicted)')
    #plt.xlabel('$t_{1/2}$ (real)')
    #plt.suptitle(f"{plot_title}")
    #plt.savefig(os.path.join(project_results_graphs, plot_title))
    #plt.show()

In [78]:
# opakovat pre rozne strojove metody, porovnan chyby
#to display graphs, un-comment all rows in previous cell starting with plot or plt
type_ml_use = 'linear'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type linear
MAE train: 0.2470
MAE test: 0.7197


C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.672e-02, tolerance: 6.966e-03
  model = cd_fast.enet_coordinate_descent(


In [79]:
type_ml_use = 'KRR'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type KRR
MAE train: 0.2274
MAE test: 0.6352


In [80]:
type_ml_use = 'RF'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type RF
MAE train: 0.3392
MAE test: 0.5554


In [81]:
type_ml_use = 'GB'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type GB
MAE train: 0.2144
MAE test: 0.3618


In [82]:
type_ml_use = 'ANN'
reg, plot_title = model_choice(type_ml_use)
mach_learn(x_train, x_test, y_train, y_test, reg, plot_title)

using ML type ANN
MAE train: 0.2984
MAE test: 0.4429
